In [2]:
# standard data science packages
import numpy as np
import pandas as pd

# imodels imports
from imodels.tree.rf_plus.rf_plus.rf_plus_models import \
    RandomForestPlusRegressor, RandomForestPlusClassifier
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import \
    RFPlusMDI, AloRFPlusMDI

# functions for subgroup experiments
from subgroup_detection import *
from subgroup_experiment import *
import shap

# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, \
    accuracy_score, r2_score, f1_score, log_loss, root_mean_squared_error

# pipeline imports
from subgroup import *

In [3]:
# set inputs
seed = 1
dataids = [361247, 361243, 361242, 361251, 361253, 361260, 361259, 361256, 361254, 361622]
dataid = 361616
clustertype = "kmeans"

In [4]:
# get data
X, y = get_openml_data(dataid)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    test_size = 0.5,
                                                    random_state=seed)

# check if task is regression or classification
if len(np.unique(y)) == 2:
    task = 'classification'
else:
    task = 'regression'
    
print("Step 1")
    
# fit the prediction models
rf, rf_plus_baseline, rf_plus = fit_models(X_train, y_train, task)

rf_plus_ridge = RandomForestPlusRegressor(rf_model=rf,
                                            prediction_model=RidgeCV(cv=5))
rf_plus_ridge.fit(X_train, y_train)

rf_plus_lasso = RandomForestPlusRegressor(rf_model=rf,
                                            prediction_model=LassoCV(cv=5,
                                            max_iter=10000, random_state=0))
rf_plus_lasso.fit(X_train, y_train)

print("Step 2")

# obtain shap feature importances
shap_explainer = shap.TreeExplainer(rf)
shap_train_values, shap_train_rankings = get_shap(X_val, shap_explainer,
                                                    task)
shap_test_values, shap_test_rankings = get_shap(X_test, shap_explainer,
                                                task)

print("Step 3")

# get lime feature importances
lime_train_values, lime_train_rankings = get_lime(X_val, rf, task)
lime_test_values, lime_test_rankings = get_lime(X_test, rf, task)

print("Step 4")

# create list of lmdi variants
lmdi_variants = create_lmdi_variant_map()

# obtain lmdi feature importances
lmdi_explainers = get_lmdi_explainers(rf_plus, lmdi_variants,
                                        rf_plus_baseline = rf_plus_baseline,
                                        rf_plus_lasso = rf_plus_lasso,
                                        rf_plus_ridge = rf_plus_ridge)
lfi_train_values, lfi_train_rankings = get_lmdi(X_train, y_train,
                                                lmdi_variants,
                                                lmdi_explainers)
lfi_train_values, lfi_train_rankings = get_lmdi(X_val, None,
                                                lmdi_variants,
                                                lmdi_explainers)
lfi_test_values, lfi_test_rankings = get_lmdi(X_test, None,
                                                lmdi_variants,
                                                lmdi_explainers)

print("Step 5")

# add shap to the dictionaries
lfi_train_values["shap"] = shap_train_values
lfi_train_rankings["shap"] = shap_train_rankings
lfi_test_values["shap"] = shap_test_values
lfi_test_rankings["shap"] = shap_test_rankings

# add the raw data to the dictionaries as a baseline of comparison
lfi_train_values["rawdata"] = X_val
lfi_test_values["rawdata"] = X_test

# add lime to the dictionaries
lfi_train_values["lime"] = lime_train_values
lfi_test_values["lime"] = lime_test_values
    
# get the clusterings
# method_to_labels, method_to_indices = get_train_clusters(lfi_train_values, clustertype)
train_clusters = get_train_clusters(lfi_train_values, clustertype)
cluster_centroids = get_cluster_centroids(lfi_train_values, train_clusters)
test_clusters = get_test_clusters(lfi_test_values, cluster_centroids)

print("Step 6")

# compute the performance
metrics_to_scores = compute_performance(X_val, X_test, y_val, y_test,
                                        train_clusters, test_clusters, task)

print("Step 7")


/tmp/ipykernel_407309/2136635497.py:2: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  X, y = get_openml_data(dataid)


Step 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   10.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.4s finished


Step 2
Step 3
Step 4
Step 5
Step 6


/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/scratch/users/zachrewolinski/conda/envs/mdi/lib

Step 7
